# Task
We want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Echo a number and redirect it to results.txt 
    
Step 2.
1. Duplicate the content in msg.txt two times and redirect it to a new file
    
2. Get the value in results.txt and times the number by 2 and redirect it to results.txt

##### note: we used `ShellOPTemplate` to complete the task above. In this tutorial, we will achieve the same task using `PythonOPTemplate` 

import necessary packages

In [12]:
from pathlib import Path

from dflow import Step, Workflow, download_artifact, upload_artifact
from dflow.python import (OP, OPIO, Artifact, OPIOSign, PythonOPTemplate,
                          upload_packages)
from typing import List

if "__file__" in locals():
    upload_packages.append(__file__)
    
from dflow import s3_config
s3_config["endpoint"] = "127.0.0.1:60001"

For step 1: 

In [13]:
class WriteFile(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'msg': str,
            'number': int,
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "out_art": Artifact(List[Path])
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        f = open("msg.txt","w")
        f.write(op_in["msg"])
        f.close()
        g = open("results.txt","w")
        g.write(str(op_in["number"]))
        g.close()
        
        op_out = OPIO({
            "out_art": [Path("msg.txt"), Path("results.txt")],
        })

        return op_out

For Step 2

In [37]:
class Duplicate(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            "in_art": Artifact(Path),
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "out_art": Artifact(List[Path])
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        import os
        os.system(f"ls {op_in["in_art"]}")
        f = open(op_in["msg"],"r")
        content = f.read()
        f.close()
        f = open("bar.txt","w")
        f.write(content * 2)
        f.close()

        g = open(op_in["results"],"r")
        content = int(g.read())
        g.close()
        g = open("results.txt","w")
        g.write(content*2)
        g.close()
        
        op_out = OPIO({
            "out_art": [Path("bar.txt"), Path("results.txt")],
        })

        return op_out

SyntaxError: invalid syntax (2089271328.py, line 22)

In [29]:
wf=Workflow(name="python-nhcv7")
wf.query_step(name="step")[0]
#wf.query_step(name="python-nhcv7-3998726336")

RuntimeError: Workflow ID is None

In [30]:
step0 = Step(
    name="step0",
    template=PythonOPTemplate(WriteFile, image="python:3.8"),
    parameters={"msg": "HelloWorld!", "number": 1},
)

# step1 = Step(
#     name="step1",
#     template=PythonOPTemplate(Duplicate, image="python:3.8"),
#     artifacts={"msg":step0.outputs.artifacts["out_art"][0],"results":step0.outputs.artifacts["out_art"][1]}
# )
wf = Workflow(name="python")
wf.add(step0)
#wf.add(step1)
wf.submit()

Workflow has been submitted (ID: python-sb8kt)


{'metadata': {'name': 'python-sb8kt', 'generateName': 'python-', 'namespace': 'argo', 'uid': '1ac7239d-a175-4ac1-b7bf-847a3a29b8b9', 'resourceVersion': '13178', 'generation': 1, 'creationTimestamp': '2022-07-19T09:14:20Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2022-07-19T09:14:20Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'python-steps', 'inputs': {}, 'outputs': {}, 'metadata': {}, 'steps': [[{'name': 'step0', 'template': 'writefile-d4x1p', 'arguments': {'parameters': [{'name': 'msg', 'value': 'HelloWorld!', 'description': '{"type": "<class \'str\'>"}'}, {'name': 'number', 'value': '1', 'description': '{"type": "<class \'int\'>"}'}], 'artifacts': [{'name': 'dflow_python

In [36]:
wf.query_step(name="step0")[0].outputs.artifacts["out_art"]

{'name': 'out_art', 'path': '/tmp/outputs/artifacts/out_art', 's3': {'key': 'python-sb8kt/python-sb8kt-2785520844/out_art.tgz'}}